In [1]:
import pandas as pd

mmgMD = pd.read_csv('mammographic_masses.data.txt', header = None, na_values = '?')
mmgMD.head()

,0,1,2,3,4,5
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


In [2]:
column_names = ["BI_RADS", "Age", "Shape", "Margin", "Density", "Severity"]
mmgMD.columns = column_names
mmgMD.head()

,BI_RADS,Age,Shape,Margin,Density,Severity
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


In [3]:
mmgMD.describe()

,BI_RADS,Age,Shape,Margin,Density,Severity
count,959.000000,956.000000,930.000000,913.000000,885.000000,961.000000
mean,4.348279,55.487448,2.721505,2.796276,2.910734,0.463059
std,1.783031,14.480131,1.242792,1.566546,0.380444,0.498893
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,45.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


In [4]:
#We can drop the rows that contains NaN value, since the data seems randomly distributed
mmgMD[mmgMD.isna().any(axis = 1)]

,BI_RADS,Age,Shape,Margin,Density,Severity
1,4.0,43.0,1.0,1.0,NaN,1
4,5.0,74.0,1.0,5.0,NaN,1
5,4.0,65.0,1.0,NaN,3.0,0
6,4.0,70.0,NaN,NaN,3.0,0
7,5.0,42.0,1.0,NaN,3.0,0
...,...,...,...,...,...,...
778,4.0,60.0,NaN,4.0,3.0,0
819,4.0,35.0,3.0,NaN,2.0,0
824,6.0,40.0,NaN,3.0,4.0,1
884,5.0,NaN,4.0,4.0,3.0,1


In [5]:
mmgMD = mmgMD.dropna()
mmgMD.head()

,BI_RADS,Age,Shape,Margin,Density,Severity
0,5.0,67.0,3.0,5.0,3.0,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
8,5.0,57.0,1.0,5.0,3.0,1
10,5.0,76.0,1.0,4.0,3.0,1


In [6]:
Features    = mmgMD[['Age', 'Shape', 'Margin', 'Density']].values
Class       = mmgMD['Severity'].values

FeatureName =['Age', 'Shape', 'Margin', 'Density']

In [7]:
#Normalize data since the scale of age is totally different with other features
from sklearn.preprocessing import StandardScaler

Scaler = StandardScaler()
FeaturesScaled = Scaler.fit(Features)
FeaturesScaled = Scaler.transform(Features)

FeaturesScaled

array([[ 0.7650629 ,  0.17563638,  1.39618483,  0.24046607],
       [ 0.15127063,  0.98104077,  1.39618483,  0.24046607],
       [-1.89470363, -1.43517241, -1.157718  ,  0.24046607],
       ...,
       [ 0.56046548,  0.98104077,  1.39618483,  0.24046607],
       [ 0.69686376,  0.98104077,  1.39618483,  0.24046607],
       [ 0.42406719,  0.17563638,  0.11923341,  0.24046607]])

## Neural Network

In [8]:
from sklearn.model_selection import cross_val_score
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

In [9]:
def create_model():
    model = Sequential()
    model.add(Dense(128, input_dim = 4, kernel_initializer = 'normal', activation = 'relu' ))
    model.add(Dropout(0.5))
    model.add(Dense(32, kernel_initializer = 'normal', activation = 'relu' ))
    model.add(Dropout(0.5))
    model.add(Dense(1, kernel_initializer = 'normal', activation = 'sigmoid' ))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model


In [10]:
estimator = KerasClassifier(build_fn = create_model, epochs = 100, verbose = 0)

Scores = cross_val_score(estimator,FeaturesScaled, Class, cv = 10)
Scores.mean()

C:\Users\Jody Angelo\AppData\Local\Temp\ipykernel_19824\620359884.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn = create_model, epochs = 100, verbose = 0)


0.8108433663845063